In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="3" # only use this/these gpus
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import torch

!nvidia-smi
torch.cuda.get_device_name()


/home/ubuntu/miniconda3/envs/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tue Jul  2 13:17:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1B.0 Off |                    0 |
|  0%   35C    P0              68W / 300W |   5490MiB / 23028MiB |     22%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

'NVIDIA A10G'

In [2]:
# load model and tokenizer

model = AutoModelForCausalLM.from_pretrained(
    '../cense_csv/models/Llama-2-13b-hf-conversion-cense',
    #'../cense_csv/models/gemma-2-9b-conversion-cense',

    load_in_4bit=True,
    temperature = 0
)

tokenizer = AutoTokenizer.from_pretrained(
    '../cense_csv/models/Llama-2-13b-hf-conversion-cense'
    #'../cense_csv/models/gemma-2-9b-conversion-cense',
    ) 

tokenizer.pad_token = tokenizer.eos_token 
eos_token_id = tokenizer.eos_token_id

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
/home/ubuntu/miniconda3/envs/env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ubuntu/miniconda3/envs/env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation

In [3]:
# load data and generate target

# for dataset in data directory
for file_name in os.listdir('data/'):
    file_name = file_name.split('.')[0]
    print(file_name)

    # load and preprocess dataset
    df = pd.read_csv(f'data/{file_name}.csv')
    df['file_name'] = file_name

    list_of_dicts = df.to_dict(orient='records')

    # for row in dataset generate target 
    for dict in list_of_dicts:

        text = str({'file_name': file_name, 'source': dict})
        text = text+ ", 'target':"
        print('row: \n', text)

        batch = tokenizer(text, return_tensors='pt')
        input_ids = batch.to('cuda')

        with torch.cuda.amp.autocast():
            output_tokens = model.generate(**input_ids, max_new_tokens=250, num_beams=1)#, eos_token_id=stop_token_id)
        
        result = tokenizer.decode(output_tokens[0], skip_special_tokens=True).split('\n')[0]

        print('generated: \n', result)
        print()
    print("#################")

transfersFile-0
row: 
 {'file_name': 'transfersFile-0', 'source': {'Email': 'nils.burde@gmail.com', 'Date (UTC)': '2022-05-02 14:06:41', 'Transaction ID': 'ba2760d9-40b1-4396-876b-96ef38b6fe1a', 'Type': 'Transfer from spot to wallet', 'Amount': 14.65337013, 'Fee': '-', 'Transaction hash': nan, 'Main account balance': 29.16113598, 'Currency': 'CRPT', 'file_name': 'transfersFile-0'}}, 'target':


/home/ubuntu/miniconda3/envs/env/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


generated: 
 {'file_name': 'transfersFile-0', 'source': {'Email': 'nils.burde@gmail.com', 'Date (UTC)': '2022-05-02 14:06:41', 'Transaction ID': 'ba2760d9-40b1-4396-876b-96ef38b6fe1a', 'Type': 'Transfer from spot to wallet', 'Amount': 14.65337013, 'Fee': '-', 'Transaction hash': nan, 'Main account balance': 29.16113598, 'Currency': 'CRPT', 'file_name': 'transfersFile-0'}}, 'target': {'Date': '05/02/2022 14:06:41', 'inBuyAmount': 14.65337013, 'inBuyAsset': 'CRPT', 'outSellAmount': nan, 'outSellAsset': nan, 'transactionType_raw': 'Transfer from spot to wallet', 'transactionType': 'deposit', 'comments': nan, 'transactionId': 'ba2760d9-40b1-4396-876b-96ef38b6fe1a', 'operationId': nan, 'feeAmount': nan, 'feeAsset': nan}}

row: 
 {'file_name': 'transfersFile-0', 'source': {'Email': 'nils.burde@gmail.com', 'Date (UTC)': '2022-04-12 10:59:32', 'Transaction ID': '81a35392-8382-4e97-82ff-8d09de87de4d', 'Type': 'Withdraw', 'Amount': 16.634467, 'Fee': '-', 'Transaction hash': nan, 'Main account ba

KeyboardInterrupt: 

In [3]:
# for row in dataset generate target

# for dataset in data directory
for file_name in os.listdir('data/'):
    file_name = file_name.split('.')[0]
    print(file_name)

    # load and preprocess dataset
    df = pd.read_csv(f'data/{file_name}.csv')
    df['file_name'] = file_name

    list_of_dicts = df.to_dict(orient='records')

    
print(list_of_dicts[:3]) 

for dict in list_of_dicts[:3]:

    text = str({'file_name': file_name, 'source': dict})
    text = text+ ", 'target':"
    print('row: \n', text)

    batch = tokenizer(text, return_tensors='pt')
    input_ids = batch.to('cuda')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**input_ids, max_new_tokens=250, num_beams=1)#, eos_token_id=stop_token_id)
    
    result = tokenizer.decode(output_tokens[0], skip_special_tokens=True).split('\n')[0]

    print('generated: \n', result)
    print()


transfersFile-0
tradesFile-0 (2)
[{'Uuid': '5e505b2b-4213-43d6-83ca-812f1a27c481', 'Exchange': 'BTC-LTC', 'Closed (UTC)': '2017-12-05 13:57:52Z', 'Opened (UTC)': '2017-12-05 13:57:52Z', 'Type': 'LIMIT_SELL', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.00864551, 'Quantity': 79.4241529, 'Remaining': 0.0, 'Price': 0.6866623, 'Avg. Price per Share': 0.00864551, 'file_name': 'tradesFile-0 (2)'}, {'Uuid': 'cdbe50bc-2b3f-4f85-a164-4a01713e7634', 'Exchange': 'ETH-LTC', 'Closed (UTC)': '2017-12-05 13:16:21Z', 'Opened (UTC)': '2017-12-05 13:16:21Z', 'Type': 'LIMIT_BUY', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.22199999, 'Quantity': 79.4241529, 'Remaining': 0.0, 'Price': 17.63216114, 'Avg. Price per Share': 0.22199999, 'file_name': 'tradesFile-0 (2)'}, {'Uuid': '05bc699a-0f9d-4ee9-b92a-e9ab6cfa31ee', 'Exchange': 'BTC-ETH', 'Closed (UTC)': '2017-12-05 13:08:57Z', 'Opened (UTC)': '2017-12-05 13:08:57Z', 'Type': 'LIMIT_BUY', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.0

/home/ubuntu/miniconda3/envs/env/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


generated: 
 {'file_name': 'tradesFile-0 (2)', 'source': {'Uuid': '5e505b2b-4213-43d6-83ca-812f1a27c481', 'Exchange': 'BTC-LTC', 'Closed (UTC)': '2017-12-05 13:57:52Z', 'Opened (UTC)': '2017-12-05 13:57:52Z', 'Type': 'LIMIT_SELL', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.00864551, 'Quantity': 79.4241529, 'Remaining': 0.0, 'Price': 0.6866623, 'Avg. Price per Share': 0.00864551, 'file_name': 'tradesFile-0 (2)'}}, 'target': {'Date': '12/05/2017 13:57:52', 'inBuyAmount': 0.6866623, 'inBuyAsset': 'LTC', 'outSellAmount': 0.00864551, 'outSellAsset': 'BTC', 'transactionType_raw': 'LIMIT_SELL', 'transactionType': 'order', 'comments': nan, 'transactionId': nan, 'operationId': '5e505b2b-4213-43d6-83ca-812f1a27c481', 'feeAmount': nan, 'feeAsset': nan}}

row: 
 {'file_name': 'tradesFile-0 (2)', 'source': {'Uuid': 'cdbe50bc-2b3f-4f85-a164-4a01713e7634', 'Exchange': 'ETH-LTC', 'Closed (UTC)': '2017-12-05 13:16:21Z', 'Opened (UTC)': '2017-12-05 13:16:21Z', 'Type': 'LIMIT_BUY', 'Time In For

In [5]:
# batched generation 

# for dataset in data directory
for file_name in os.listdir('data/'):
    file_name = file_name.split('.')[0]
    print(file_name)

    # load and preprocess dataset
    df = pd.read_csv(f'data/{file_name}.csv')
    df['file_name'] = file_name

    list_of_dicts = df.to_dict(orient='records')


input_texts = []

for row in list_of_dicts:

    text = str(row)
    text = text+ ", 'target':" 
    input_texts.append(text)

    #batch = tokenizer(text, return_tensors='pt')
    #input_ids.append(batch.to('cuda'))

print(list_of_dicts[:3])    

input_texts = input_texts[:3]
input_ids = tokenizer(input_texts, return_tensors='pt', padding=True).input_ids.to('cuda')

# Generate text using beam search
with torch.cuda.amp.autocast():
    outputs = model.generate(input_ids, num_beams=1, max_length=250)

# Decode the generated texts
generated_texts = [tokenizer.decode(output, skip_special_tokens=True).split('\n')[0] for output in outputs]

for i in generated_texts:
    print(i)

transfersFile-0
tradesFile-0 (2)
[{'Uuid': '5e505b2b-4213-43d6-83ca-812f1a27c481', 'Exchange': 'BTC-LTC', 'Closed (UTC)': '2017-12-05 13:57:52Z', 'Opened (UTC)': '2017-12-05 13:57:52Z', 'Type': 'LIMIT_SELL', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.00864551, 'Quantity': 79.4241529, 'Remaining': 0.0, 'Price': 0.6866623, 'Avg. Price per Share': 0.00864551, 'file_name': 'tradesFile-0 (2)'}, {'Uuid': 'cdbe50bc-2b3f-4f85-a164-4a01713e7634', 'Exchange': 'ETH-LTC', 'Closed (UTC)': '2017-12-05 13:16:21Z', 'Opened (UTC)': '2017-12-05 13:16:21Z', 'Type': 'LIMIT_BUY', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.22199999, 'Quantity': 79.4241529, 'Remaining': 0.0, 'Price': 17.63216114, 'Avg. Price per Share': 0.22199999, 'file_name': 'tradesFile-0 (2)'}, {'Uuid': '05bc699a-0f9d-4ee9-b92a-e9ab6cfa31ee', 'Exchange': 'BTC-ETH', 'Closed (UTC)': '2017-12-05 13:08:57Z', 'Opened (UTC)': '2017-12-05 13:08:57Z', 'Type': 'LIMIT_BUY', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.0

In [7]:
for i in generated_texts:
    print(i)

{'Uuid': '5e505b2b-4213-43d6-83ca-812f1a27c481', 'Exchange': 'BTC-LTC', 'Closed (UTC)': '2017-12-05 13:57:52Z', 'Opened (UTC)': '2017-12-05 13:57:52Z', 'Type': 'LIMIT_SELL', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.00864551, 'Quantity': 79.4241529, 'Remaining': 0.0, 'Price': 0.6866623, 'Avg. Price per Share': 0.00864551, 'file_name': 'tradesFile-0 (2)'}, 'target': {'Date': '12/05/2017 13:57:52', 'inBuyAmount': 0.6866623, 'inBuyAsset': 'LTC', 'outSellAmount': 0.00864551, 'outSellAsset': 'BTC', 'transactionType_raw': 'LIMIT_SELL', 'transactionType': 'order', 'comments': 'IMMEDIATE_OR_CANCEL', 'transactionId': nan, 'operationId': '5e505b2b-4213-43d6-83ca-812f1a27c481', 'feeAmount': nan, 'feeAsset': nan}}
{'Uuid': 'cdbe50bc-2b3f-4f85-a164-4a01713e7634', 'Exchange': 'ETH-LTC', 'Closed (UTC)': '2017-12-05 13:16:21Z', 'Opened (UTC)': '2017-12-05 13:16:21Z', 'Type': 'LIMIT_BUY', 'Time In Force': 'IMMEDIATE_OR_CANCEL', 'Bid/Ask': 0.22199999, 'Quantity': 79.4241529, 'Remaining': 0.0,